In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import jv, jvp, yv, iv
from scipy.optimize import root_scalar
plt.rcParams.update({'font.size': 22})
%matplotlib qt

global kp, cP, cS, radius

def Viktorov_matrix(omega, p , cl, ct, R):
    '''
    Equation I.28 can be written as a matrix.  Where the determinant of that matrix is zero, we have the roots
    of the equality.
    omega = frequency
    p = angular wavenumber
    cl = longitudinal wave velocity of material
    ct = shear wave velocity of material
    '''
    kl = omega/cl
    kt = omega/ct
    x = kl*R
    y = kt*R
    # Matrix form
    a = (jv(p+2,x) + jv(p-2,x) - 2*((kt/kl)**2 -1)*jv(p,x))
    b = (jv(p+2,y) - jv(p-2,y))
    c = (jv(p+2,x) - jv(p-2,x))
    d = (jv(p+2,y) + jv(p-2,y))
    M = np.array([[a, b],[c, d]])
    return M

def D_det(omega): 
    '''Calculate the determinant of matrix '''
    return np.linalg.det(Viktorov_matrix(omega, kp, cP, cS, radius))

def VpVs(poisson):
    return np.sqrt((2*(1-poisson))/(1-2*poisson))

# DISPERSION OF ELASTIC WAVES AND RAYLEIGH-TYPE WAVES IN A CYLINDER

This notebook summarizes the calculations of Viktorov (1958), Equation (I.28).
  
In this notebook, for a practical implementation, we use a $V_P = 5660~m/s$ which corresponds to steel.  The radius of the cylinder is $6.28~cm$ .

In [2]:
# CONSTANTS
r1 = 0.0628 #0.0127#0.02225
r = r1
C = 2*np.pi*r1
cL = 5660 #6350 #3480 #5660 # m/s
cT = 3120 #3170 #cL/2.03100960115899 #cL/2#3120 # m/s
cR = 2932.9
f = 1.0*1e6 # Hz
omega = 2*np.pi*f
wl = cL/f
c = omega*wl/(2*np.pi)
kp = 2*np.pi*r1/wl # kappa (angular wavenumber - dimensionless)
phi = cT/cL

print('Circumference %3.2f mm' % (C*1000))
print('wavelength: %3.2f mm' % (wl*1000))
print('Speed (c):  %3.2f m/s' %  c)
print('Angular wavenumber (kp): %3.2f' % kp)
print('Vs/Vp (phi):  %3.2f' % phi)

Circumference 394.58 mm
wavelength: 5.66 mm
Speed (c):  5660.00 m/s
Angular wavenumber (kp): 69.71
Vs/Vp (phi):  0.55


## Re-scaling of values

By dividing the velocities by a $1000$ and multiplying the radius by the same factor, the velocity units are expressed in $mm/\mu s$ and the radius in $mm$.

In [3]:
# Re-scale the values
cS = cT/1000
cP = cL/1000
cR = cR/1000
radius = 1000*r1

## Solve for the roots of the determinant 

The system of equations can be written as a 2x2 matrix.  The condition for non-trivial solutions requires: 

$$det(D(k,w))$$

For each $k$ value along all the frequencies considered, the function $det(D(k,w))$ cancels out for a set of values of $w$ which are the roots of the characteristic equation of the matrix. 

The roots are found by the bisection method.

In [4]:
%%time
# Define constants and range of calculations
fstop = 5.0 # Ending frequency in MHz
dp = 0.2 # 0.1 
wstart = 0.001; 
dw = 0.01 #0.001; #0.01
kpstart = 0.001 #0.04*radius; #0.01
kpstop = 20 #20.0#20.0 #kstop*radius;

omegas = np.arange(wstart,2*np.pi*fstop,dw)
kps = np.arange(kpstart,kpstop,dp)

number_of_roots = 10 #10 

roots = []
KP = []
for kp in kps: #kp 
    print(kp)    
    rc = 0
    for omega in omegas: 
        if kp <= 1:
            NR = number_of_roots - 1 #
            if rc < NR: #number_of_roots:                
                if (np.sign(D_det(omega)) > np.sign(D_det(omega+dw))) or (np.sign(D_det(omega)) < np.sign(D_det(omega+dw))):
                    #print(omega)\
                    root_val = root_scalar(D_det, bracket = [omega,omega+dw], method = 'bisect', xtol = 1e-7)# root_val.root                    
                    if rc == 0:
                        roots.append(0)
                        KP.append(kp)
                    if root_val != None:                        
                        rc = rc + 1                                           
                        roots.append(root_val.root)
                        KP.append(kp)
                        print(kp,root_val.root)
            elif rc == NR:
                break
        elif kp > 1:
            NR = number_of_roots #
            if rc < NR: #number_of_roots:
                if (np.sign(D_det(omega)) > np.sign(D_det(omega+dw))) or (np.sign(D_det(omega)) < np.sign(D_det(omega+dw))):
                    #print(omega)\
                    root_val = root_scalar(D_det, bracket = [omega,omega+dw], method = 'bisect', xtol = 1e-7)# root_val.root                    
                    if root_val != None:
                        rc = rc + 1
                        roots.append(root_val.root)
                        KP.append(kp)
                        print(kp,root_val.root)
                        if rc == 1:
                            wstart = root_val.root
                            omegas = np.arange(wstart,2*np.pi*fstop,dw)
            elif rc == NR:
                break


0.001
0.001 0.01775437927246094
0.001 0.18985276794433592
0.001 0.25522935485839837
0.001 0.41826173400878897
0.001 0.48746202087402346
0.001 0.577370620727539
0.001 0.7351641998291014
0.001 0.7736830291748045
0.001 0.8923497161865234
0.201
0.201 0.07252198791503905
0.201 0.22071336364746091
0.201 0.2719542083740234
0.201 0.43408708190917966
0.201 0.5159266815185546
0.201 0.5931273040771483
0.201 0.7508017120361328
0.201 0.8019963226318357
0.201 0.9079956207275389
0.401
0.401 0.09189118957519529
0.401 0.248341079711914
0.401 0.2889329681396484
0.401 0.4495970306396484
0.401 0.5437025604248045
0.401 0.6087845001220701
0.401 0.766273208618164
0.401 0.8299177703857419
0.401 0.9235484466552732
0.6010000000000001
0.6010000000000001 0.10829667663574218
0.6010000000000001 0.27305360412597657
0.6010000000000001 0.3067849884033203
0.6010000000000001 0.464879165649414
0.6010000000000001 0.5707881317138672
0.6010000000000001 0.6244146881103514
0.6010000000000001 0.7816143951416015
0.6010000000000

In [5]:
#%matplotlib notebook
plt.figure()
plt.scatter(KP,roots)

In [6]:
roots = np.array(roots)
KPvals = np.unique(KP)
if number_of_roots > 1:    
    ROOTS = roots.reshape((len(KPvals),number_of_roots))
    KPvals = KPvals#[1:]
else:
    ROOTS = roots    

# Plot the dispersion curves 

In [7]:
k_ = KPvals
w_ = ROOTS*radius/cS

In [8]:
if number_of_roots ==1:
    plt.figure()
    # plt.plot(KPvals,ROOTS, label = 'mode 1')
    plt.plot(k_,w_, label = 'mode 1')
    plt.xlabel('kp')
    plt.ylabel(r'$\omega$ (x 1e6)')
    #plt.axis([1.0,np.max(KP),0,np.max(roots)])
    plt.legend()
    plt.grid(True)
else:
    plt.figure()
    # plt.plot(KPvals,ROOTS[:,0], label = 'mode 1')
    # plt.plot(KPvals,ROOTS[:,1], label = 'mode 2')
    # plt.plot(KPvals,ROOTS[:,2], label = 'mode 3')
    # plt.plot(KPvals,ROOTS[:,3], label = 'mode 4')
    # plt.plot(KPvals,ROOTS[:,4], label = 'mode 5')
    # plt.plot(KPvals,ROOTS[:,5], label = 'mode 6')
    # plt.plot(KPvals,ROOTS[:,6], label = 'mode 7')
    # plt.plot(KPvals,ROOTS[:,7], label = 'mode 8')
    # plt.plot(KPvals,ROOTS[:,8], label = 'mode 9')
    # plt.plot(KPvals,ROOTS[:,9], label = 'mode 10')
    plt.plot(k_,w_[:,0], label = 'mode 1')
    plt.plot(k_,w_[:,1], label = 'mode 2')
    plt.plot(k_,w_[:,2], label = 'mode 3')
    plt.plot(k_,w_[:,3], label = 'mode 4')
    plt.plot(k_,w_[:,4], label = 'mode 5')
    plt.plot(k_,w_[:,5], label = 'mode 6')
    plt.plot(k_,w_[:,6], label = 'mode 7')
    plt.plot(k_,w_[:,7], label = 'mode 8')
    plt.plot(k_,w_[:,8], label = 'mode 9')
    plt.plot(k_,w_[:,9], label = 'mode 10')
    plt.xlabel(r'$\bar{k}$')
    plt.ylabel(r'$\bar{\omega}$')
    #plt.axis([1.0,np.max(KP),0,np.max(roots)])
    plt.legend()
    plt.grid(True)

In [9]:
if number_of_roots ==1:
    mode1 = 1000*w_*cS/k_
    plt.figure()
    plt.title('Velocity dispersion')
    plt.plot(KPvals, mode1, label = 'mode 1')
    plt.xlabel('kp')
    plt.ylabel('c (m/s)')
    plt.axis([0.0,np.max(KP),0, 8000.0])
    plt.legend()
    plt.grid(True)
else:
    
    mode1 = 1000*w_[:,0]*cS/k_
    mode2 = 1000*w_[:,1]*cS/k_
    mode3 = 1000*w_[:,2]*cS/k_
    mode4 = 1000*w_[:,3]*cS/k_
    mode5 = 1000*w_[:,4]*cS/k_
    mode6 = 1000*w_[:,5]*cS/k_

    # mode1 = (ROOTS[:,0]*radius/cR)/KPvals
    # mode2 = (ROOTS[:,1]*radius/cR)/KPvals
    # mode3 = (ROOTS[:,2]*radius/cR)/KPvals
    # mode4 = (ROOTS[:,3]*radius/cR)/KPvals
    # mode5 = (ROOTS[:,4]*radius/cR)/KPvals
    # mode6 = (ROOTS[:,5]*radius/cR)/KPvals

    plt.figure()
    plt.title('Velocity dispersion')
    plt.plot(k_, mode1, label = 'mode 1')
    plt.plot(k_, mode2, label = 'mode 2')
    plt.plot(k_, mode3, label = 'mode 3')
    plt.plot(k_, mode4, label = 'mode 4')
    plt.plot(k_, mode5, label = 'mode 5')
    plt.plot(k_, mode6, label = 'mode 6')
    plt.axhline(0.9214,lw=0.5,ls='--',c='r')
    plt.xlabel(r'$\bar{k}$')
    plt.ylabel('c (m/s)')
    plt.axis([0.0,np.max(KP),0, 8000.0])
    plt.legend()
    plt.grid(True)

In [10]:
if number_of_roots ==1:
    mode1 = w_/k_#[1:]
    plt.figure()
    plt.title('Relative velocity dispersion')
    plt.plot(k_, mode1, label = 'mode 1')
    plt.xlabel(r'$\bar{k}$')
    plt.ylabel('c/cT')
    plt.axis([0.0,np.max(k_),0, 4.0])
    plt.legend()
    plt.grid(True)
else:
    mode1 = w_[:,0]/k_
    mode2 = w_[:,1]/k_
    mode3 = w_[:,2]/k_
    mode4 = w_[:,3]/k_
    mode5 = w_[:,4]/k_
    mode6 = w_[:,5]/k_
    mode7 = w_[:,6]/k_
    mode8 = w_[:,7]/k_
    mode9 = w_[:,8]/k_
    mode10 = w_[:,9]/k_
    
    # mode1 = (ROOTS[:,0]*radius/cR)/KPvals#[1:]
    # mode2 = (ROOTS[:,1]*radius/cR)/KPvals#[1:]
    # mode3 = (ROOTS[:,2]*radius/cR)/KPvals#[1:]
    # mode4 = (ROOTS[:,3]*radius/cR)/KPvals#[1:]
    # mode5 = (ROOTS[:,4]*radius/cR)/KPvals#[1:]
    # mode6 = (ROOTS[:,5]*radius/cR)/KPvals#[1:]
    # mode7 = (ROOTS[:,6]*radius/cR)/KPvals#[1:]
    # mode8 = (ROOTS[:,7]*radius/cR)/KPvals#[1:]
    # mode9 = (ROOTS[:,8]*radius/cR)/KPvals#[1:]
    # mode10 = (ROOTS[:,9]*radius/cR)/KPvals#[1:]
    
    plt.figure()
    plt.title('Relative velocity dispersion')
    plt.plot(k_, mode1, label = 'mode 1')
    plt.plot(k_, mode2, label = 'mode 2')
    plt.plot(k_, mode3, label = 'mode 3')
    plt.plot(k_, mode4, label = 'mode 4')
    plt.plot(k_, mode5, label = 'mode 5')
    plt.plot(k_, mode6, label = 'mode 6')
    plt.plot(k_, mode7, label = 'mode 7')
    plt.plot(k_, mode8, label = 'mode 8')
    plt.plot(k_, mode9, label = 'mode 9')
    plt.plot(k_, mode10, label = 'mode 10')
    #plt.axhline(0.9214,lw=0.5,ls='--',c='r')
    plt.xlabel(r'$\bar{k}$')
    plt.ylabel('c/cT')
    plt.axis([0.0,np.max(k_),0, 4.0])
    plt.legend()
    plt.grid(True)

In [11]:
# Matrix = np.c_[k_, mode1, mode2, mode3, mode4, mode5, mode6, mode7, mode8, mode9, mode10 ]
# np.savetxt('Steel_Dispersion_Viktorov__Vel_Rel.mod', Matrix, delimiter = ',')

In [12]:
# Matrix = np.c_[k_, w_]
# np.savetxt('Steel_Dispersion_Viktorov.mod', Matrix, delimiter = ',')

# Calculate the determinant for all omegas and kps

This part of the notebook is just to illustrate how the bisection method correctly found the roots of the determinant.

In [13]:
# Check all iterations of determinant matrix
DD = []
for kp in kps:
    for omega in omegas:
        DD.append(np.linalg.det(Viktorov_matrix(omega, kp, cP, cS, radius)))        
DD = np.array(DD).reshape((len(kps),len(omegas)))    

## Plot a particular kp value and the roots

In [14]:
m = 37#298
s = 0 # mode
print(kps[m])
# Plot the function and the roots for a particular kp
plt.figure()
plt.title('kp: '+str(kps[m])+'')
plt.plot(omegas,DD[m,:]/np.max(np.abs(DD[m,:])))
plt.scatter(ROOTS[m,:],np.zeros(np.size(ROOTS[m,:])))
plt.grid(True)

7.401000000000001


In [15]:
DD.shape

(100, 3044)

In [16]:
#m = 50
print(m)
D_root = Viktorov_matrix(ROOTS[m,s], kps[m], cP, cS, radius)
print('Matrix of root')
print(D_root)
print('This determinant should be zero: ', np.linalg.det(D_root))

37
Matrix of root
[[ 0.05743263 -0.14729981]
 [-0.13817473  0.35438239]]
This determinant should be zero:  2.1787994262349564e-09


# Get the coefficients A and D for a root

In order to do that, we solve the system of equations for a unit amplitude for radial displacements.  The solution gives the relative relationship between A and D.

$$D_{root} \begin{bmatrix} A \\ D \end{bmatrix} = \begin{bmatrix} 1 \\ 0 \end{bmatrix}$$

In [17]:
s = 0

In [18]:
A0 = np.zeros((2,1))

A0[0,0]=1.0
print(A0)

[[1.]
 [0.]]


In [19]:
x1 = np.linalg.inv(D_root.T@D_root)@(D_root.T@A0)
print(x1)

[[1.61475070e+08]
 [6.29596037e+07]]


In [20]:
n1=x1/x1[0,0]
# Coefficients
A = n1[0]
D = n1[1]
print(A)
print(D)

[1.]
[0.38990293]


In [21]:
# Another possible normalization
# n2 = x1/np.abs(np.sum(x1))
# print(n2)
# A = n2[0]
# D = n2[1]
# print(A)
# print(D)

# Calculate the radial displacement

In [22]:
m = 38
kp = kps[m]
print(kp)
r1 = radius
R = np.linspace(0.001,radius,100)#0.5*radius

print(c)
phi = cS/cP

u_r = []
u_theta = []
omega_root = ROOTS[m,s] 
for r in R:
    # Radial displacement. Equation 21, page 92
    #arg =-A*(omega_root/cP)*(cP**2/omega_root**2)*jvp(kp,omega_root*r/cP,1)-1j*2*D*((cS**2*kp)/(r*omega_root**2))*jv(kp,omega_root*r/cS)
    arg_r =-A*(cP/omega_root)*jvp(kp,omega_root*r/cP,1)-1j*2*D*((cS**2*kp)/(r*omega_root**2))*jv(kp,omega_root*r/cS)
    # Tangential displacement. Equation 21, page 92
    arg_theta =(1j*A*((kp*cP**2)/(r*omega_root**2))*jv(kp,omega_root*r/cP)-2*D*(cS**2/omega_root**2)*(omega_root/cS)*jvp(kp,omega_root*r/cS,1))
    u_r.append(arg_r) 
    u_theta.append(-arg_theta) # Maybe a typo in paper? had to multiply the value by -1 to match figures
    
u_r = np.array(u_r)
u_theta = np.array(u_theta)

7.601000000000001
5660.0


In [23]:
plt.figure()
plt.plot(R/R.max(),np.real(u_r)/u_r[-1], label =r'$u_r/\hat{u}_{r}$')
plt.plot(R/R.max(),np.imag(u_theta)/u_r[-1], label =r'$u_\theta/\hat{u}_\theta}_{r}$')
plt.xlabel(r'$r/r_1$')
plt.ylabel('Normalized Amplitude')
#plt.gca().invert_xaxis()
plt.grid(True)
plt.legend()

/home/leo/anaconda3/envs/jrudge/lib/python3.10/site-packages/matplotlib/cbook/__init__.py:1335: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)


In [24]:
m

38

In [25]:
cP/cS

1.814102564102564

In [26]:
np.sqrt(3)/3

0.5773502691896257

In [27]:
VpVs(0.25)

1.7320508075688772

# Calculate and plot radial and tangential displacements for a particular kp

In [28]:
%%time
def GetCoefficients(D_root): 
    # Set the displacement u_r to unity
    A0 = np.zeros((2,1))
    A0[0,0]=1.0
    #print(A0)
    # Solve the system of equations
    x1 = np.linalg.inv(D_root.T@D_root)@(D_root.T@A0)
    print(x1)
    # Normalize 
    n1=x1/x1[0,0]
    # Store Coefficients
    A = n1[0]
    D = n1[1]
    print(A)
    print(D)
    return A,D

# Define constants and range of calculations
fstop = 5.0 # Ending frequency in MHz
wstart = 0.001; 
dw = 0.01#0.001#0.001; #0.01
omegas = np.arange(wstart,2*np.pi*fstop,dw)

# Calculate the roots for a particular value of kp oover all frequencies
kp = 5 #200 # p as in Destrade and Fu (2008)
root_kp =[]
for omega in omegas: 
    if (np.sign(D_det(omega)) > np.sign(D_det(omega+dw))) or (np.sign(D_det(omega)) < np.sign(D_det(omega+dw))):
        root_val = root_scalar(D_det, bracket = [omega,omega+dw], method = 'bisect', xtol = 1e-7)# root_val.root      
    if root_val != None:
        root_kp.append(root_val.root)
        #print(kp,root_val)

# Check the root of a particular mode s        
s = 0 # Mode
print('Root for mode s: ',root_kp[s])

# Estimate the matrix for that specific root
D_root = Viktorov_matrix(root_kp[s], kp, cP, cS, radius)
print(D_root)

# Solve for the eigenvector coefficients
[A,D] = GetCoefficients(D_root)

# Calculate the radial displacements
print(kp)
r1 = radius
R = np.linspace(0.001,radius,1000)#0.5*radius

print(c)
phi = cS/cP

u_r = []
u_theta = []
omega_root = root_kp[s]#ROOTS[m,s] 
for r in R:
    # Radial displacement. Equation 21, page 92
    #arg =-A*(omega_root/cP)*(cP**2/omega_root**2)*jvp(kp,omega_root*r/cP,1)-1j*2*D*((cS**2*kp)/(r*omega_root**2))*jv(kp,omega_root*r/cS)
    arg_r =-A*(cP/omega_root)*jvp(kp,omega_root*r/cP,1)-1j*2*D*((cS**2*kp)/(r*omega_root**2))*jv(kp,omega_root*r/cS)
    # Tangential displacement. Equation 21, page 92
    arg_theta =(1j*A*((kp*cP**2)/(r*omega_root**2))*jv(kp,omega_root*r/cP)-2*D*(cS**2/omega_root**2)*(omega_root/cS)*jvp(kp,omega_root*r/cS,1))
    u_r.append(arg_r) 
    u_theta.append(-arg_theta) # Maybe a typo in paper? had to multiply the value by -1 to match figures
    
u_r = np.array(u_r)
u_theta = np.array(u_theta)

# PLot the displacements
plt.figure()
plt.title('Mode s = %i' % s)
plt.plot(R/R.max(),np.real(u_r)/u_r[-1], label =r'$u_r/\hat{u}_{r}$')
plt.plot(R/R.max(),-np.imag(u_theta)/u_r[-1], label =r'$u_\theta/\hat{u}_\theta}_{r}$')
plt.xlabel(r'$r/r_1$')
plt.ylabel('Normalized Amplitude')
#plt.gca().invert_xaxis()
plt.grid(True)
plt.legend()

Root for mode s:  2.5465364227294933
[[-0.32979833 -0.03854867]
 [ 0.1004914  -0.10269877]]
[[-2.72095181]
 [-2.66246879]]
[1.]
[0.97850641]
5
5660.0
CPU times: user 524 ms, sys: 50.9 ms, total: 575 ms
Wall time: 500 ms


# Same as before using Scipy's root_scalar bisection method

Using the in-built bisection function inside `scipy.optimize`.

In [29]:
%%time
# def D_determinant(omega, kp, cP, cS, radius):#, cP, cS, radius): 
#     '''Calculate the determinant of matrix '''
#     return np.real(np.linalg.det(D_matrix(omega, kp, cP, cS, radius)))

def D_determinant(omega, kp, cP, cS, radius):#, cP, cS, radius): 
    '''Calculate the determinant of matrix '''
    return np.real(np.linalg.det(Viktorov_matrix(omega, kp, cP, cS, radius)))

# Define constants and range of calculations
fstop = 5.0 # Ending frequency in MHz
wstart = 0.001; 
dw = 0.01#0.001; #0.01
omegas = np.arange(wstart,2*np.pi*fstop,dw)

# Calculate the roots for a particular value of kp oover all frequencies
kp = 10 # p as in Destrade and Fu (2008)
root_kp = [] # Roots for a particular kp value
for omega in omegas:               
    fa = D_determinant(omega,kp, cP, cS, radius) # Evaluate function at location a of [a,b]
    fb = D_determinant(omega+dw,kp, cP, cS, radius) # Evaluate function at location b of [a,b]
    if (fa > 0) == (fb < 0): # Checks that the function has opposite signs in interval [a,b]
        root_val=root_scalar(D_determinant,args = (kp, cP, cS, radius),method='bisect',bracket=[omega,omega+dw], rtol =1e-7)
        if (root_val.root > omega):# (sol.root > m):
            root_kp.append(root_val.root)            
            
# Check the root of a particular mode s        
s = 0 # Mode
print('Root for mode s: ',root_kp[s])

# Estimate the matrix for that specific root
D_root = Viktorov_matrix(root_kp[s], kp, cP, cS, radius)
print(D_root)

# Solve for the eigenvector coefficients
[A,D] = GetCoefficients(D_root)

# Calculate the radial displacements
print(kp)
r1 = radius
R = np.linspace(0.001,radius,1000)#0.5*radius

print(c)
phi = cS/cP

u_r = []
u_theta = []
omega_root = root_kp[s]#ROOTS[m,s] 
for r in R:
    # Radial displacement. Equation 21, page 92
    #arg =-A*(omega_root/cP)*(cP**2/omega_root**2)*jvp(kp,omega_root*r/cP,1)-1j*2*D*((cS**2*kp)/(r*omega_root**2))*jv(kp,omega_root*r/cS)
    arg_r =-A*(cP/omega_root)*jvp(kp,omega_root*r/cP,1)-1j*2*D*((cS**2*kp)/(r*omega_root**2))*jv(kp,omega_root*r/cS)
    # Tangential displacement. Equation 21, page 92
    arg_theta =(1j*A*((kp*cP**2)/(r*omega_root**2))*jv(kp,omega_root*r/cP)-2*D*(cS**2/omega_root**2)*(omega_root/cS)*jvp(kp,omega_root*r/cS,1))
    u_r.append(arg_r) 
    u_theta.append(-arg_theta) # Maybe a typo in paper? had to multiply the value by -1 to match figures
    
u_r = np.array(u_r)
u_theta = np.array(u_theta)

# PLot the displacements
plt.figure()
plt.title('Mode s = %i' % s)
plt.plot(R/R.max(),np.real(u_r)/u_r[-1], label =r'$u_r/\hat{u}_{r}$')
plt.plot(R/R.max(),-np.imag(u_theta)/u_r[-1], label =r'$u_\theta/\hat{u}_\theta}_{r}$')
plt.xlabel(r'$r/r_1$')
plt.ylabel('Normalized Amplitude')
#plt.gca().invert_xaxis()
plt.grid(True)
plt.legend()

Root for mode s:  0.5265953598022461
[[ 0.02324194 -0.1795008 ]
 [-0.04802626  0.37091378]]
[[2.03043297e+08]
 [2.62902367e+07]]
[1.]
[0.12948094]
10
5660.0
CPU times: user 354 ms, sys: 87.8 ms, total: 442 ms
Wall time: 351 ms
